In [1]:
#!python -m spacy download pt_core_news_lg
#!python -m spacy download en_core_web_lg
#%pip install -U scikit-learn

In [1]:
import os
import io
import stardog
import pandas as pd
import spacy
from spacy import displacy
from sklearn.cluster import DBSCAN
import numpy as np
import requests
from unidecode import unidecode
import re
import json
from langdetect import detect

### Specify Stardog connection details

In [2]:
# Stardog variables
STARDOG_ENDPOINT = os.getenv('STARDOG_ENDPOINT')
STARDOG_USERNAME = os.getenv("STARDOG_USERNAME")
STARDOG_PASSWORD = os.getenv("STARDOG_PASSWORD")

connection_details = {
    'endpoint': STARDOG_ENDPOINT,
    'username': STARDOG_USERNAME,
    'password': STARDOG_PASSWORD
}

In [4]:
database_name = 'IndigenousSlavery'
conn = stardog.Connection(database_name, **connection_details)

In [5]:
# Query que busca as teses e os seus abstracts. Foi incluído um filtro para buscar apenas 
# os abstract em português, inglês ou em outra língua
query = """
SELECT ?thesis ?abstract (lang(?abstract) AS ?lang) WHERE {
  ?thesis a <http://purl.org/ontology/bibo/Thesis>.
  ?thesis <http://purl.org/ontology/bibo/abstract> ?abstract.
  FILTER (lang(?abstract) IN("pt", "en"))
  
}

"""

query2 = """
SELECT ?thesis ?ent  WHERE {
  ?thesis a <http://purl.org/ontology/bibo/Thesis>.
  ?thesis <https://schema.org/mentions> ?ent.  
}

"""

query3 = """
SELECT ?place   WHERE {
  ?place a <https://schema.org/Place>. 
}

"""
# FILTER (langMatches(lang(?abstract),"pt"))
# FILTER (langMatches(lang(?abstract),"en"))
# FILTER (lang(?abstract) NOT IN("pt", "en"))
# LIMIT 20

csv_results = conn.select(query, content_type='text/csv')
thesis_abstract = pd.read_csv(io.BytesIO(csv_results))
thesis_abstract


,thesis,abstract,lang
0,tag:stardog:api:en_an_abolicionist_argument_pe...,[pt] O presente trabalho busca fazer uma retom...,pt
1,tag:stardog:api:percepcoes_dos_pesquisadores_d...,NaN,pt
2,tag:stardog:api:com_a_condicao_de_servir_gratu...,A presente pesquisa parte da emancipação de es...,pt
3,tag:stardog:api:entre_as_pessoas_mais_gradas_d...,O objeto central que baliza esta pesquisa é a ...,pt
4,tag:stardog:api:saraiva_dantas_e_cotegipe_baia...,A presente tese investiga o baianismo e suas i...,pt
...,...,...,...
22705,tag:stardog:api:micropropagacao_fotoautotrofic...,The success of mass micropropagation of fruit ...,en
22706,tag:stardog:api:contrastes_e_continuidades_em_...,NaN,pt
22707,tag:stardog:api:assombro_de_olhada_de_bicho_um...,This ethnography has as an objective to compre...,en
22708,tag:stardog:api:o_devirclandestino_em_marcados...,In the dissertation O devir-clandestino em Mar...,en


In [6]:
# Extraíndo as entidades dos abstracts

#Carregando os modelos SpaCy para inglês e português 
nlp_en = spacy.load("en_core_web_lg")
nlp_pt = spacy.load("pt_core_news_lg")

#Dicionário que receberá as instâncias de cada tese
instances_dic = {}

for n in range(len(thesis_abstract)):

    # Processando os abstracts em português
    if thesis_abstract['lang'][n] == 'pt':

        doc_pt = nlp_pt(str(thesis_abstract['abstract'][n]))
        persons = []
        gpes = []
        #orgs = []
        for ent in doc_pt.ents:
            if ent.label_ == "PER": #"PERSON":
                persons.append(ent)
            if ent.label_ == "LOC": #"GPE":
                gpes.append(ent)
            #if ent.label_ == "ORG":
            #    orgs.append(ent)

        #instances_dic[thesis_abstract['thesis'][n].replace('tag:stardog:api:','')] = {'PER':persons, 'LOC':gpes, 'ORG':orgs, 'lang': 'pt'}
        instances_dic[thesis_abstract['thesis'][n].replace('tag:stardog:api:','')] = {'PER':persons, 'LOC':gpes, 'lang': 'pt'}

    # Processando os abstracts em inglês
    if thesis_abstract['lang'][n] == 'en':

        doc_en = nlp_en(str(thesis_abstract['abstract'][n]))
        persons = []
        gpes = []
        #orgs = []
        for ent in doc_en.ents:
            if ent.label_ == "PERSON":
                persons.append(ent)
            if ent.label_ == "GPE":
                gpes.append(ent)
            #if ent.label_ == "ORG":
            #    orgs.append(ent)

        #instances_dic[thesis_abstract['thesis'][n].replace('tag:stardog:api:','')] = {'PER':persons, 'LOC':gpes, 'ORG':orgs, 'lang': 'en'}
        instances_dic[thesis_abstract['thesis'][n].replace('tag:stardog:api:','')] = {'PER':persons, 'LOC':gpes, 'lang': 'en'}

    if n%500 == 499:
        print(n, " teses processadas.")
    #    break

499  teses processadas.
999  teses processadas.
1499  teses processadas.
1999  teses processadas.
2499  teses processadas.
2999  teses processadas.
3499  teses processadas.
3999  teses processadas.
4499  teses processadas.
4999  teses processadas.
5499  teses processadas.
5999  teses processadas.
6499  teses processadas.
6999  teses processadas.
7499  teses processadas.
7999  teses processadas.
8499  teses processadas.
8999  teses processadas.
9499  teses processadas.
9999  teses processadas.
10499  teses processadas.
10999  teses processadas.
11499  teses processadas.
11999  teses processadas.
12499  teses processadas.
12999  teses processadas.
13499  teses processadas.
13999  teses processadas.
14499  teses processadas.
14999  teses processadas.
15499  teses processadas.
15999  teses processadas.
16499  teses processadas.
16999  teses processadas.
17499  teses processadas.
17999  teses processadas.
18499  teses processadas.
18999  teses processadas.
19499  teses processadas.
19999  t

In [7]:
# Funcão que recebe uma string e a limpar para ficar no formato aceitável para uma URI
def process_uri(x):
    return (re.sub('[^a-zA-Z0-9_ ]', '',
            unidecode(x.strip())
                        .replace(" ", "_")
                        .replace("[","")
                        .replace("]","")
                        .replace("?","")
                        .replace("'","")
                        .lower()))

In [8]:
# Criando URIs para as entidades
for thes in instances_dic:
    for type_ent in ['LOC', 'PER']:
        list_ent = []
        for span in instances_dic[thes][type_ent]:
            list_ent.append(process_uri(span.text))
        instances_dic[thes][type_ent + '_uri'] = list(set(list_ent))

Preparando o texto em turtle

In [9]:
# Prefixos
prefixos = """ @prefix ns: <http://www.w3.org/2003/06/sw-vocab-status/ns#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix bibo: <http://purl.org/ontology/bibo/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix vann: <http://purl.org/vocab/vann/> .
@prefix event: <http://purl.org/NET/c4dm/event.owl#> .
@prefix prism: <http://prismstandard.org/namespaces/1.2/basic/> .
@prefix terms: <http://purl.org/dc/terms/> .
@prefix schema: <http://schemas.talis.com/2005/address/schema#> .
@prefix status: <http://purl.org/ontology/bibo/status/> .
@prefix degrees: <http://purl.org/ontology/bibo/degrees/> .
@prefix stardog: <tag:stardog:api:> .
@base <http://www.w3.org/2002/07/owl#> .

"""

Conectando com o banco e adicionando as triplas

In [10]:
# Funcão que recebe os prefixos e triplas e as carrega à base de dados
def add_triplas_to_stardog(prefixos, triplas):

    # Incluindo prefixos às triplas
    triplas = prefixos + " " + triplas

    ### Connect to the Stardog database
    database_name = 'IndigenousSlavery'
    conn = stardog.Connection(database_name, **connection_details)

    conn.begin()
    conn.add(stardog.content.Raw(triplas, 'text/turtle'))
    conn.commit() # commit the transaction

Preparando as triplas para conectar entidades às teses e carregando à base.

In [11]:
loc_uri = []
for thes in instances_dic:
    for loc in instances_dic[thes]['LOC_uri']:
        loc_uri.append(loc)

triplas = """ """
for loc in (set(loc_uri)):
    triplas = triplas + " " + """
    stardog:""" + loc + """ rdf:type <https://schema.org/Place>."""

add_triplas_to_stardog(prefixos, triplas)

In [12]:
per_uri = []
for thes in instances_dic:
    for per in instances_dic[thes]['PER_uri']:
        per_uri.append(per)

triplas = """ """
for per in (set(per_uri)):
    triplas = triplas + " " + """
    stardog:""" + per + """ rdf:type foaf:Person."""

add_triplas_to_stardog(prefixos, triplas)

In [13]:
triplas = """ """
n = 0
for thes in instances_dic:
    n = n + 1
    for loc in instances_dic[thes]['LOC_uri']:
        tripla = """
        stardog:""" + thes + """ <https://schema.org/mentions> stardog:""" + loc + """.
        """
        triplas = triplas + " " + tripla

    for per in instances_dic[thes]['PER_uri']:
        tripla = """
        stardog:""" + thes + """ <https://schema.org/mentions> stardog:""" + per + """.
        """  
        triplas = triplas + " " + tripla
    
add_triplas_to_stardog(prefixos, triplas)